# Feature Engineering

---

1. Import packages
2. Load data
3. Feature engineering

---

## 1. Import packages

In [7]:
import pandas as pd

---
## 2. Load data

In [8]:
df = pd.read_csv('./clean_data_after_eda.csv')
df["date_activ"] = pd.to_datetime(df["date_activ"], format='%Y-%m-%d')
df["date_end"] = pd.to_datetime(df["date_end"], format='%Y-%m-%d')
df["date_modif_prod"] = pd.to_datetime(df["date_modif_prod"], format='%Y-%m-%d')
df["date_renewal"] = pd.to_datetime(df["date_renewal"], format='%Y-%m-%d')

In [9]:
df.head(3)

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,var_6m_price_off_peak_var,var_6m_price_peak_var,var_6m_price_mid_peak_var,var_6m_price_off_peak_fix,var_6m_price_peak_fix,var_6m_price_mid_peak_fix,var_6m_price_off_peak,var_6m_price_peak,var_6m_price_mid_peak,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,0.000131,4.100838e-05,0.000908,2.086294,99.530517,44.235794,2.086425,9.953056e+01,44.236702,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,0.000003,1.217891e-03,0.000000,0.009482,0.000000,0.000000,0.009485,1.217891e-03,0.000000,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,0.000004,9.450150e-08,0.000000,0.000000,0.000000,0.000000,0.000004,9.450150e-08,0.000000,0


---

## 3. Feature engineering

### Difference between off-peak prices in December and preceding January

Below is the code created by your colleague to calculate the feature described above. Use this code to re-create this feature and then think about ways to build on this feature to create features with a higher predictive power.

In [10]:
price_df = pd.read_csv('price_data (1).csv')
price_df["price_date"] = pd.to_datetime(price_df["price_date"], format='%Y-%m-%d')
price_df.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [11]:
# Group off-peak prices by companies and month
monthly_price_by_id = price_df.groupby(['id', 'price_date']).agg({'price_off_peak_var': 'mean', 'price_off_peak_fix': 'mean'}).reset_index()

# Get january and december prices
jan_prices = monthly_price_by_id.groupby('id').first().reset_index()
dec_prices = monthly_price_by_id.groupby('id').last().reset_index()

# Calculate the difference
diff = pd.merge(dec_prices.rename(columns={'price_off_peak_var': 'dec_1', 'price_off_peak_fix': 'dec_2'}), jan_prices.drop(columns='price_date'), on='id')
diff['offpeak_diff_dec_january_energy'] = diff['dec_1'] - diff['price_off_peak_var']
diff['offpeak_diff_dec_january_power'] = diff['dec_2'] - diff['price_off_peak_fix']
diff = diff[['id', 'offpeak_diff_dec_january_energy','offpeak_diff_dec_january_power']]
diff.head()

,id,offpeak_diff_dec_january_energy,offpeak_diff_dec_january_power
0,0002203ffbb812588b632b9e628cc38d,-0.006192,0.162916
1,0004351ebdd665e6ee664792efc4fd13,-0.004104,0.177779
2,0010bcc39e42b3c2131ed2ce55246e3c,0.050443,1.500000
3,0010ee3855fdea87602a5b7aba8e42de,-0.010018,0.162916
4,00114d74e963e47177db89bc70108537,-0.003994,-0.000001


Now it is time to get creative and to conduct some of your own feature engineering! Have fun with it, explore different ideas and try to create as many as yo can!

In [12]:
from datetime import datetime

# Convert date columns to datetime
date_columns = ['date_activ', 'date_end', 'date_modif_prod', 'date_renewal']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# 1. Date-Based Features
# Extract year, month, and day of the week from dates
for col in date_columns:
    df[f'{col}_year'] = df[col].dt.year
    df[f'{col}_month'] = df[col].dt.month
    df[f'{col}_day_of_week'] = df[col].dt.dayofweek

# Duration features (e.g., number of days since activation, last modification, etc.)
df['days_since_activation'] = (df['date_end'] - df['date_activ']).dt.days
df['days_since_last_modif'] = (df['date_end'] - df['date_modif_prod']).dt.days
df['days_until_renewal'] = (df['date_renewal'] - df['date_end']).dt.days

# 2. Aggregation and Transformation Features
# Consumption ratios
df['monthly_to_yearly_cons_ratio'] = df['cons_last_month'] / (df['cons_12m'] + 1e-5)  # Avoid division by zero
df['gas_to_energy_ratio'] = df['cons_gas_12m'] / (df['cons_12m'] + 1e-5)

# Forecast accuracy (difference between forecast and actual)
df['forecast_accuracy_cons'] = df['forecast_cons_12m'] - df['cons_12m']
df['forecast_accuracy_discount'] = df['forecast_discount_energy'] - df['cons_last_month']

# 3. Interaction Features
# Interaction between energy price and forecast consumption
df['price_peak_forecast_interaction'] = df['forecast_price_energy_peak'] * df['forecast_cons_12m']
df['price_off_peak_forecast_interaction'] = df['forecast_price_energy_off_peak'] * df['forecast_cons_12m']
df['cons_energy_gas_interaction'] = df['cons_12m'] * df['cons_gas_12m']

# 4. Encoding Categorical Variables
# One-Hot Encoding for categorical variables
df = pd.get_dummies(df, columns=['channel_sales', 'origin_up', 'has_gas'], drop_first=True)

# 5. Lagged Features and Trends (Assuming 'cons_last_month' df is time-ordered)
# Rolling average of last 3 months' consumption
df['cons_last_3_months_avg'] = df['cons_last_month'].rolling(window=3, min_periods=1).mean()

# 6. Outlier Flags
# Flagging potential outliers in consumption and pricing variables (custom thresholds)
df['high_energy_consumption'] = (df['cons_12m'] > df['cons_12m'].quantile(0.95)).astype(int)
df['high_gas_consumption'] = (df['cons_gas_12m'] > df['cons_gas_12m'].quantile(0.95)).astype(int)

# 7. Statistical Features for Price Variance
price_columns = [col for col in df.columns if 'price' in col]
df['price_var_mean'] = df[price_columns].mean(axis=1)
df['price_var_std'] = df[price_columns].std(axis=1)
df['price_var_range'] = df[price_columns].max(axis=1) - df[price_columns].min(axis=1)

# Display the updated df with new features
df.head()


,id,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,...,origin_up_ldkssxwpmemidmecebumciepifcamkci,origin_up_lxidpiddsbxsbosboudacockeimpuepw,origin_up_usapbepcfoloekilkwsdiboslwaxobdp,has_gas_t,cons_last_3_months_avg,high_energy_consumption,high_gas_consumption,price_var_mean,price_var_std,price_var_range
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,...,False,True,False,True,0.000000,0,0,20.777382,31.143269,99.530558
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,...,False,False,False,False,0.000000,0,0,3.137846,10.666687,44.311378
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,...,False,False,False,False,0.000000,0,0,2.467284,9.304249,44.311378
3,bba03439a292a1e166f80264c16191cb,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,...,False,False,False,False,0.000000,0,0,3.464498,11.535266,44.311378
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,...,False,False,False,False,175.333333,0,0,5.979567,15.853515,52.108175


In [14]:

# Group off-peak prices by company ID and month
monthly_price_by_id = df.groupby(['id', df['date_end'].dt.to_period('M')]).agg({
    'var_6m_price_off_peak_var': 'mean',
    'var_6m_price_off_peak_fix': 'mean'
}).reset_index()
monthly_price_by_id.columns = ['id', 'price_date', 'price_off_peak_var', 'price_off_peak_fix']

# Extract January and December prices
jan_prices = monthly_price_by_id[monthly_price_by_id['price_date'].dt.month == 1].groupby('id').first().reset_index()
dec_prices = monthly_price_by_id[monthly_price_by_id['price_date'].dt.month == 12].groupby('id').first().reset_index()

# Merge and calculate the December-to-January off-peak price difference
seasonal_diff = pd.merge(dec_prices[['id', 'price_off_peak_var', 'price_off_peak_fix']], 
                         jan_prices[['id', 'price_off_peak_var', 'price_off_peak_fix']], 
                         on='id', suffixes=('_dec', '_jan'))
seasonal_diff['offpeak_diff_dec_jan_energy'] = seasonal_diff['price_off_peak_var_dec'] - seasonal_diff['price_off_peak_var_jan']
seasonal_diff['offpeak_diff_dec_jan_power'] = seasonal_diff['price_off_peak_fix_dec'] - seasonal_diff['price_off_peak_fix_jan']

# Keep only the relevant columns
seasonal_diff = seasonal_diff[['id', 'offpeak_diff_dec_jan_energy', 'offpeak_diff_dec_jan_power']]

# Merge the seasonal difference back to the main DataFrame
df = pd.merge(df, seasonal_diff, on='id', how='left')
df.head()


,id,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,...,cons_last_3_months_avg,high_energy_consumption,high_gas_consumption,price_var_mean,price_var_std,price_var_range,offpeak_diff_dec_jan_energy_x,offpeak_diff_dec_jan_power_x,offpeak_diff_dec_jan_energy_y,offpeak_diff_dec_jan_power_y
0,24011ae4ebbe3035111d65fa7c15bc57,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,...,0.000000,0,0,20.777382,31.143269,99.530558,NaN,NaN,NaN,NaN
1,d29c2c54acc38ff3c0614d0a653813dd,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,...,0.000000,0,0,3.137846,10.666687,44.311378,NaN,NaN,NaN,NaN
2,764c75f661154dac3a6c254cd082ea7d,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,...,0.000000,0,0,2.467284,9.304249,44.311378,NaN,NaN,NaN,NaN
3,bba03439a292a1e166f80264c16191cb,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,...,0.000000,0,0,3.464498,11.535266,44.311378,NaN,NaN,NaN,NaN
4,149d57cf92fc41cf94415803a877cb4b,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,...,175.333333,0,0,5.979567,15.853515,52.108175,NaN,NaN,NaN,NaN
